# Basics on how to use the tool for MPC

In [ ]:
# set path for acados
import os
ld_library_path = os.environ.get('LD_LIBRARY_PATH')
if ld_library_path is not None:
    print(f'LD_LIBRARY_PATH: {ld_library_path}')
else:
    print('LD_LIBRARY_PATH is not set.')

acados_source_path = os.environ.get('ACADOS_SOURCE_DIR')
if acados_source_path is not None:
    print(f'ACADOS_SOURCE_DIR: {acados_source_path}')
else:
    print('ACADOS_SOURCE_DIR is not set.')

import torch
import numpy as np

In [ ]:
from src.mpc_classes_acados import AMPC_class, get_AMPC_trajectory

from src.inverted_pendulum import nonlinear_pend_dynamics, F_model_RK4
from src.mpc_classes import MPC_class, get_MPC_trajectory

from src.parameters import MPC_Param, AMPC_Param
from src.plotting import plot_MPC_results

from src.utils import *
from src.means import get_Base_acados_MPC_means

from bokeh.io import output_notebook, show
output_notebook()

curr_path = os.getcwd()

In [ ]:
RESULTS_DIR = os.path.abspath('Results')
MPC_DATASETS_DIR = os.path.join(RESULTS_DIR, 'MPC_data_gen')
MPC_RESULTS_DIR = os.path.join(RESULTS_DIR, 'CMPC_results')

## Set initial parameter values

In [ ]:
N_short = 8     # length of the short MPC horizon
N_long  = 30    # length of the long MPC horizon

NUM_MPCS = 10

mpc_params = {
'T_sim': 5, # length of the closed-loop simulation (in seconds)
}

# Parameters are stored in a special python dataclass, that is defined in src/parameters:
P = AMPC_Param(N_MPC=N_long,**mpc_params)

In [ ]:
# The dataclass is basically a dictionary with easier way to access the variables:
print(P,P.Ts,P.N_sim,sep='\n')

## Load older results

In [ ]:
MPC_results = []
# # load base result
# file_name = 'MPC_outcome_Tutorial_07_11_23_19_20.pkl' # 'MPC_outcome_Tutorial.pkl', 'Mean_MPC_outcome_all.pkl', 'MPC_outcome_Tutorial_07_11_23_19_20.pkl'
# file_path = os.path.join(curr_path, 'Results', file_name)
# loaded_MPC_results = load_results(file_path)

# base_MPC_name = f'MPC({N_long})'
# MPC_results[base_MPC_name] = loaded_MPC_results[base_MPC_name]

# # load other results
# file_name = 'MPC_outcome_Tutorial.pkl' # 'MPC_outcome_Tutorial.pkl', 'Mean_MPC_outcome_all.pkl', 'MPC_outcome_Tutorial_07_11_23_19_20.pkl'
# file_path = os.path.join(curr_path, 'Results', file_name)
# loaded_MPC_results = load_results(file_path)
# MPC_results.update(loaded_MPC_results)

## Generating closed-loop trajectories for standard MPC

In [ ]:
# Just for the ease of use I package all controllers as a dictionary with keys used as names for plotting later
standard_controllers = {
    f'MPC({N_long})': MPC_class(P),
    f'MPC({N_short})': MPC_class(MPC_Param(N_MPC=N_short,**mpc_params))
}

In [ ]:
# To run the closed loop with the controller we can loop over the dictionary and store each controller's outcome as another dictionary with the same keys
for version in range(NUM_MPCS):
    for cname,cMPC in standard_controllers.items():
        # The Fmodel is only used for simulating the plant, so you can use a different set of parameters to emulate model-plant mismatch
        model = F_model_RK4(nonlinear_pend_dynamics(cMPC.P), cMPC.P)
        # The function that generates the closed loop is called get_MPC_trajectory.
        # It uses the parameter [P.N_sim] in the controller to decide how long the trajectory will be
        results = get_MPC_trajectory(cMPC, model, cname=cname)
        file_path = os.path.join(MPC_RESULTS_DIR, f'CMPC_results_{cMPC.P.N_MPC}M_{version}v.ph')
        results.save(file_path)

## Generating closed-loop trajectories for acados MPC

In [ ]:
# qp_solver
# ---------
#       'FULL_CONDENSING_QPOASES', 'PARTIAL_CONDENSING_HPIPM', 'FULL_CONDENSING_HPIPM', 
#       'PARTIAL_CONDENSING_QPDUNES', 'PARTIAL_CONDENSING_OSQP', 'FULL_CONDENSING_DAQP'
# hessian_approx
# --------------
#       'GAUSS_NEWTON', 'EXACT'
# integrator_type
# ---------------
#       'ERK', 'IRK', 'DISCRETE', 'LIFTED_IRK', 'GNSF'
# nlp_solver_type
# ---------------
#       'SQP_RTI', 'SQP'
# regularize_method
# -----------------
#       'NO_REGULARIZE', 'MIRROR', 'PROJECT', 'PROJECT_REDUC_HESS', 'CONVEXIFY'
# hpipm_mode
# ----------
#       'BALANCE', 'SPEED_ABS', 'SPEED', 'ROBUST'
# collocation_type
# ----------------
#       'GAUSS_RADAU_IIA', 'GAUSS_LEGENDRE'
# globalization
# -------------
#       'FIXED_STEP', 'MERIT_BACKTRACKING'

# acados_options = get_all_acados_options(base_cname=f'AMPC({N_long})')

acados_options = {
    f'RTI_FC_HPIPM': dict(
        qp_solver='FULL_CONDENSING_HPIPM', 
        nlp_solver_type='SQP_RTI', 
        integrator_type='DISCRETE'
    ),
    f'SQP_FC_HPIPM': dict(
        qp_solver='FULL_CONDENSING_HPIPM', 
        nlp_solver_type='SQP', 
        integrator_type='DISCRETE'
    ),
    # f'AMPC({N_long})_FC_HPIPM_DISCRETE': dict(
    #     qp_solver='FULL_CONDENSING_HPIPM', 
    #     integrator_type='DISCRETE', 
    #     nlp_solver_exact_hessian=True,
    #     nlp_solver_type='SQP', 
    #     nlp_solver_tol_stat=1e-8,
    #     nlp_solver_tol_eq=1e-3
    # ),
    #     f'AMPC({N_long})_FC_HPIPM_DISCRETE_CL_RIC_NO_FACT': dict(
    #     qp_solver='FULL_CONDENSING_HPIPM', 
    #     integrator_type='DISCRETE', 
    #     nlp_solver_exact_hessian=True,
    #     nlp_solver_type='SQP', 
    #     nlp_solver_tol_stat=1e-8,
    #     nlp_solver_tol_eq=1e-3,
    #     qp_solver_ric_alg=0,
    #     qp_solver_cond_ric_alg=0
    # ),
    # f'AMPC({N_long})_SQP_PC_QPDUNES': dict(nlp_solver_type='SQP', qp_solver='PARTIAL_CONDENSING_QPDUNES'),
    # f'AMPC({N_long})_SQP': dict(nlp_solver_type='SQP'),
    f'SQP_FC_QPOASES_DISCRETE': dict(
        qp_solver='FULL_CONDENSING_QPOASES', 
        nlp_solver_type='SQP', 
        integrator_type='DISCRETE'
    ),
    
    # f'AMPC({N_long})_RTI_FC_QPOASES_DISCRETE': dict(
    #     qp_solver='FULL_CONDENSING_QPOASES', 
    #     nlp_solver_type='SQP_RTI', 
    #     integrator_type='DISCRETE'
    # ),
    # f'AMPC({N_long})_SQP_PC_QPDUNES_GN_DISCRETE': dict(
    #     qp_solver='PARTIAL_CONDENSING_QPDUNES', 
    #     integrator_type='DISCRETE', 
    #     nlp_solver_type='SQP_RTI'
    # # ),
    # f'AMPC({N_long})_PC_OSQP': dict(qp_solver='PARTIAL_CONDENSING_OSQP'),
    # f'AMPC({N_long})_RTI_FC_DAQP_GN_DISCRETE': dict(qp_solver='FULL_CONDENSING_DAQP', integrator_type='DISCRETE', nlp_solver_type='SQP_RTI'),
    # f'AMPC({N_long})_SQP_FC_DAQP_GN_IRK': dict(qp_solver='FULL_CONDENSING_DAQP', integrator_type='IRK', nlp_solver_type='SQP')
    
}

In [ ]:
for cname, coptions in acados_options.items():
    cMPC = AMPC_class(P, acados_options=coptions, acados_name=cname)
    MPC_results.append(get_AMPC_trajectory(cMPC))
    cMPC.del_solver_ocp_model()

## Generate Means of desired iterations

In [ ]:
# mean_MPC_results, MPC_results_iter = get_Base_acados_MPC_means(P, acados_options, num_rep=5)
# MPC_results.update(MPC_results_iter)

## Plot the resulting closed-loop trajectories for comparison

In [ ]:
show(plot_MPC_results(
    MPC_results, 
    plot_mpc_trajectories=False, 
    xbnd=1.5, 
    plot_solver_iterations=False,
    group_by=lambda x: (x.acados_name if isinstance(x, AMPC_data) else x.name)
))

## Get minimal settling time

In [ ]:

minimal_Ts_options = get_minimal_settling_Time(MPC_results)
print(minimal_Ts_options)

minimal_key = sorted(minimal_Ts_options[0][0].keys())[0]
minimal_time = minimal_Ts_options[1] * MPC_results[minimal_key]['P'].Ts
print(f'Minimal settling time: {minimal_time} s')
print(f'Associated acados option combination(s): \n{minimal_Ts_options[0]}')

## Get minimal mean computing time

In [ ]:
minimal_mean_time = get_minimal_mean_time(MPC_results)

print(f'Minimal mean computation time: {minimal_mean_time[1] * 1e3} ms')
print(f'Associated acados option combination(s): \n{minimal_mean_time[0]}')

## MPC trajectory differences and plot

In [ ]:
MPC_diffs = get_MPC_diffs(MPC_results, comparison_cname=f'MPC({N_long})')
show(plot_MPC_differences(MPC_diffs, plot_mpc_trajectories=False, xbnd=1.5))

## Get minimal difference to original MPC

In [ ]:
# loaded_MPC_diffs = get_MPC_diffs(loaded_MPC_results)
# print(loaded_MPC_diffs.items())
minimal_diff_options = get_minimal_difference_options(MPC_diffs)

print(f'Minimal option of maximal difference: {minimal_diff_options[1]}')
print(f'Associated acados option combination(s): \n{minimal_diff_options[0]}')

## Saving the results

In [ ]:
import pickle

result_path = os.path.join(curr_path, 'Results')
if not os.path.exists(result_path):
    os.mkdir(result_path)

file_path = os.path.join(result_path, f'MPC_outcome_Tutorial.pkl') # 'Mean_MPC_outcome_all.pkl', 'MPC_outcome_all.pkl', 'MPC_outcome_Tutorial'
with open(file_path, 'wb') as handle:
    pickle.dump(MPC_results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'Results saved in {file_path}')